# Agentic AI: The MCP Era
## How Generative AI Agents are now able to interact with the world around them!

- **Author:** Gianluca Aguzzi
- **Event:** Reading group @ Cesena - November 2025
- **Code repository:** [todo]

## Generative AI Agents?
> **Generative AI:** Machine learning models capable of generating new content based on training data (e.g., text, images, music).

> **Agents:** Autonomous entities that can perceive their environment, make decisions, and take actions to achieve specific goals.

> **Generative AI Agents:** Entities that typically use Generative AI for decision-making, problem-solving, and interaction with their environment.

## Agentic AI?
> The core concept of **Agentic AI** is the use of AI agents to perform automated tasks with limited human intervention.

- **Not completely autonomous:** Human intervention is often still required.
- **Main focus:** Automated Tasks.
  - *E.g., scheduling meetings, managing emails, data analysis, content creation.*
- **Goal:** Increase efficiency and productivity by leveraging AI capabilities.

## How we may think an AI Agent works
- Here put a simple image of an AI agent architecture

## How Do They Really Work? (Most of the time)

## LLM/Generative AI Provider
- The backbone of most generative AI agents.
- They provide a **unified interface** to interact with the model.
- **Under the hood, they handle:**
    - Model hosting
    - Model runtime
    - API services
- They often provide additional features (e.g., authentication, rate limiting, monitoring).
- **Examples:**
    - OpenAI API
    - Ollama

## Memory Management
- Agents often need to remember past interactions or context to make informed decisions.
- Memory management systems help store, retrieve, and manage this information effectively.
- **Types of memory:**
    - **Short-term memory:** Temporary storage for recent interactions (e.g., chat history).
    - **Long-term memory:** Persistent storage for important information (e.g., vector databases, SQL).

## Tool Integration
- Agents need to perform actions.
- LLMs are primarily text generators; they need a mechanism to interact with the external world.
- **Tools** are external functionalities that agents can use to perform specific tasks.
- **Examples of tools:**
    - Web browsers
    - APIs (e.g., weather API, stock market API)
    - File systems
- *Tools are essential for enabling "agentic" capabilities in AI agents.*

## What is a Tool?
- A **Tool** in this context is typically defined by:
    - A **Name**
    - A **Description**
    - **Arguments** (Schema) that the tool expects
- This structure allows the agent to understand *what* the tool does and *how* to use it.

## LLMs Without Tools
- Let's observe how an LLM behaves **without** tools.
- **Example:** Asking for the current time.
- *Spoiler: It will likely hallucinate or state its training data cutoff.*

## LangChain
- A popular framework for building applications with LLMs.
- **Provides abstractions for:**
    - Using LLMs
    - Prompt Engineering (templates, pre-built prompts)
    - Managing memory
    - Integrating tools
    - Building agents

## Tool Integration - Few Shot Learning
- Since LLMs have shown incredible performance with human-aligned instructions, we can use **few-shot learning** to teach them how to use tools.
- **The Idea:**
    1.  Inject available tool definitions into the context.
    2.  Provide examples of how to use them.
    3.  Let the LLM figure out *when* and *how* to use them.
    4.  When the LLM decides to use a tool, we parse the output, execute the tool, and provide the result back to the LLM.

## Demo: The Prompt

## Demo: Parsing the Output

## Demo: Agents with Tools

## Live Examples

## Tools in LangChain
- LangChain provides a simple way to define and use tools within agents.
- You can create a custom tool by tagging a function with the `@tool` decorator.

## Structured Tools
- Under the hood, a function tagged with `@tool` is converted into a `StructuredTool`.
- A `StructuredTool` requires:
    - **Name**
    - **Description**
    - **`args_schema`**: A Pydantic model defining the arguments the tool expects.
- This allows LangChain to automatically generate prompts and parse outputs when using the tool within an agent.

## LangChain Agents
- LangChain provides several pre-built agent types that can be easily instantiated and used.
- **Under the hood, these agents handle:**
    - Deciding when to use tools.
    - Formatting prompts.
    - Parsing tool outputs.
    - *(and much more, but we will focus on these three).*

## The Tool Dilemma
- **Problem:** I often need to implement the same tool for multiple AI agents or frameworks.
- **Fragmentation:**
    - Initially, each library/framework had its own way to define tools.
    - OpenAI introduced a standard using JSON Schema.
    - Gemini had its own proprietary format.
    - Anthropic had another.
- **Result:** This fragmentation created significant challenges for developers (the "N x M" integration problem).

## Towards Standardization
- **The Goal:**
    - Define a standard way to describe tools (name, description, arguments).
    - Use **JSON Schema** for argument definition.
    - Create a standard protocol to expose such tools to different AI agents.

## MCP: Model Context Protocol
> An open standard to define and share **context** across different AI agents and frameworks.
- **Why "Context" and not just "Tools"?**
    - While tools are a major part, other resources can be shared too.
    - **Resources:** File-like data that can be read by clients (e.g., logs, code files).
    - **Prompts:** Pre-defined templates for interacting with the server.
    - **Tools:** Executable functions.
    - In Agentic AI literature, these are collectively referred to as "context".
- MCP aims to standardize the definition and sharing of this context.

## Why Standardize?
- Just as **REST APIs** standardized web services...
- Just as **LSP (Language Server Protocol)** standardized how IDEs talk to language tools...
- **MCP** aims to standardize how AI agents and frameworks define and share context.
- **Origin:** Anthropic (creators of Claude) introduced the MCP standard to solve the ecosystem fragmentation.

## MCP Components
- **Host:** An entity that **needs** context to operate (e.g., Claude Desktop, IDEs, AI Agents).
- **Server:** An entity that **provides** context to hosts (e.g., Google Drive MCP, Postgres MCP).
- **Client:** An entity that interacts with the server on behalf of the host and manages the connection (1:1 connection).
- *This architecture allows for a "Many-to-Many" ecosystem where any Host can talk to any Server.*

## Overall Architecture


## Overall Interaction
![](./images/interaction.png)

### MCP Client - Server Interaction
- The **MCP Client** is responsible for:
    - Connecting to the MCP Server.
    - Requesting context (Resources, Prompts, Tools) on behalf of the Host.
    - Handling sampling (server asking the client for completions).
- The **MCP Server** is responsible for:
    - Exposing context to the MCP Client.
    - Handling requests for context.
    - Managing tool execution requests.

## Layers
- **Data Layer (Inner):**
  - Defines the **logic** (JSON-RPC 2.0).
  - Handles Lifecycle, Primitives (Tools, Resources, Prompts).
- **Transport Layer (Outer):**
  - Defines the **communication** (Stdio, HTTP).
  - Handles connection, message framing.
- *Data layer is transport-agnostic.*


## MCP LifeCycle
- **Stateful Protocol:** MCP requires a persistent connection state.
- **Initialization Phase:**
  - **Capability Negotiation:** Client and Server exchange supported features (e.g., "I support resources", "I support sampling").
  - **Handshake:** `initialize` request $\rightarrow$ `initialized` notification.
- **JSON-RPC 2.0:** The standard format for all messages.


## Bi-directional Communication
- It's not just Client $\rightarrow$ Server!
- **Server $\rightarrow$ Client Requests:**
  - **Sampling:** Server asks Host to generate text (using the Host's LLM).
  - **Create Message:** Server asks Host to show a message/input to user.
  - **Logging:** Server sends logs to the Host console.


## Primitives & Discovery
- **Dynamic Discovery:** Clients discover capabilities via `*/list` methods.
  - `tools/list`, `resources/list`, `prompts/list`.
- **Execution/Retrieval:**
  - `tools/call`: Execute a function.
  - `resources/read`: Get data content.
  - `prompts/get`: Get a template.


## Transport Agnosticism
- **Decoupled Architecture:** The interaction between Client and Server is independent of the transport mechanism.
- **Flexibility:** Clients and Servers operate identically whether over local STDIO or remote HTTP.
- **Development Focus:** Developers can build core logic first, then plug in the appropriate transport layer (Stdio, SSE, etc.) based on deployment needs.


## STDIO Transport
- The **STDIO Transport** uses standard input and output streams for communication.
- **How it works:**
    - The MCP Client and Server read from and write to their respective standard input/output streams.
    - Messages are serialized as JSON and exchanged over these streams.
- **Use Cases:**
    - Local development and testing.
    - Integrating MCP Servers as subprocesses in applications.
    - I want to do not make my tools available over the network!!


## SSE (Server-Sent Events) Transport
- **Mechanism:** Uses HTTP for initial connection and Server-Sent Events for server-to-client messages.
- **How it works:**
    - **Client -> Server:** Standard HTTP POST requests (for sending commands/requests).
    - **Server -> Client:** A persistent HTTP connection using SSE (for pushing updates/events).
- **Pros:**
    - **Remote Access:** Allows connecting to servers running on different machines or cloud environments.
    - **Web Compatible:** Works well with standard web infrastructure (proxies, load balancers).
- **Use Cases:**
    - Cloud-hosted agents accessing local resources via a bridge.
    - Distributed systems where agents and tools reside on different nodes.

## OK, but how to develop an MCP Server?
- There are multiple SDKs available to help you build MCP Servers quickly.
- **Python SDK:** [link]
- **Node.js SDK:** [link]
- **Go SDK:** [link]
- And many other


## FastMCP
- A fast and easy-to-use MCP Server framework in Python.
- Ok, let's make an example for time feature

## Debugging: The MCP Inspector
- Developing agents is hard; debugging them is harder.
- **MCP Inspector:** A web-based tool to test and inspect MCP Servers.
- **Capabilities:**
  - List available tools, resources, and prompts.
  - Execute tools and view raw JSON-RPC messages.
  - **Command:** `npx @modelcontextprotocol/inspector <command-to-run-server>`


## Using MCP Servers (The Client Side)
- **Ready-made Hosts:**
  - **Claude Desktop:** Native support for local MCP servers.
  - **IDEs:** VS Code (via extensions), Cursor, Zed.
- **Programmatic Clients:**
  - **LangChain / LangGraph:** Easily integrate MCP tools into custom agents.
  - **Custom Scripts:** Use the SDK to build your own client.


## The "Write Once, Run Anywhere" for AI Tools
- **Before MCP:** Write a tool for LangChain, rewrite for Semantic Kernel, rewrite for OpenAI Assistants...
- **With MCP:**
  - Write the tool **once** as an MCP Server.
  - Use it in **Claude Desktop** for manual testing/usage.
  - Use it in **VS Code** for coding assistance.
  - Use it in **Your Custom Agent** for automated workflows.


## Why This Matters for Us? (Research/Dev)
- **Scenario:** Scientific Discovery / Engineering Loop.
  - `Code` $\rightarrow$ `Simulation` $\rightarrow$ `Analysis` $\rightarrow$ `Fix`
- **Goal:** Automate this loop with Agentic AI.
- **MCP's Role:**
  - Provides a standard interface to expose our simulators and analysis tools to *any* agent.
  - Decouples the "Tooling" from the "Intelligence".


## Use Case: Alchemist Simulator
- **Context:** A simulator for aggregate computing (Swarm Intelligence).
- **MCP Server exposes:**
  - `compile(yaml_spec)`: Checks if the simulation spec is valid.
  - `simulate(yaml_spec)`: Runs the simulation and returns snapshots/metrics.
- **The Agent:** Can iteratively write the spec, fix errors, and analyze results without human intervention.


## Live demo

## Conclusion
- **MCP is the "REST API" for AI Context.**
  - Standardizes how AI models connect to data and tools.
- **Solves Fragmentation:** No more proprietary tool definitions.
- **Enables an Ecosystem:**
  - **Server Devs:** Build tools once, reach all agents.
  - **Agent Devs:** Access a massive library of existing tools.
- **Future:** Expect MCP to become the default standard for Agentic AI integration.
